In [1]:
import sqlite3

In [2]:
db_path = '/home/hyunseok/health/ehrsql-2024/data/mimic_iv/mimic_iv.sqlite'
con = sqlite3.connect(db_path)

In [3]:
cur = con.cursor()

In [48]:
res = cur.execute('''
    WITH PatientAge AS (
    SELECT DISTINCT
        age
    FROM
        admissions
    WHERE
        subject_id = 10026406
    ORDER BY
        admittime
    LIMIT 1
),
RBCResults AS (
    SELECT DISTINCT
        l.valuenum
    FROM
        labevents l
    JOIN
        admissions a ON l.hadm_id = a.hadm_id
    JOIN
        d_labitems dli ON l.itemid = dli.itemid
    WHERE
        LOWER(dli.label) = 'red blood cells'
    AND
        a.age = (SELECT age FROM PatientAge)
),
PercentileCalc AS (
    SELECT DISTINCT
        (COUNT(*) FILTER (WHERE valuenum <= 4.46) * 1.0) / COUNT(*) AS percentile
    FROM
        RBCResults
)
SELECT
    percentile
FROM
    PercentileCalc;
''')
print(res.fetchall())

[(0.9333333333333333,)]


[('patients',),
 ('admissions',),
 ('d_icd_diagnoses',),
 ('d_icd_procedures',),
 ('d_labitems',),
 ('d_items',),
 ('diagnoses_icd',),
 ('procedures_icd',),
 ('labevents',),
 ('prescriptions',),
 ('cost',),
 ('chartevents',),
 ('inputevents',),
 ('outputevents',),
 ('microbiologyevents',),
 ('icustays',),
 ('transfers',)]

In [50]:
res = cur.execute('''SELECT
    COUNT(*) AS artificial_tear_ointment_count
FROM
    prescriptions
WHERE
    LOWER(drug) = 'artificial tear ointment';
''')
print(res.fetchall())

[(3,)]


In [ ]:
con.close()